In [ ]:
import numpy as np 
import time
import datetime
import os
import sys

import yagmail
import keyring
import pyvisa

import board
import busio
import adafruit_ads1x15.ads1015 as ADS
from adafruit_ads1x15.analog_in import AnalogIn

In [ ]:
''' MAC ONLY - INTERFACE USING PYVISA & NI GPIB EXPLORER'''
# set up measurement devices
rm = pyvisa.ResourceManager()
#pyvisa.log_to_screen()
print(rm)
print(rm.list_resources())
my_instrument = rm.open_resource('GPIB0::13::INSTR')
my_instrument.write('*IDN?')
my_instrument.read()

In [ ]:
''' LINUX ONLY - INTERFACE USING LINUX-GPIB '''
import Gpib

inst = Gpib.Gpib(0,13) # Device address 0
inst.write("*IDN?")
inst.read(100)

In [ ]:
'''
    measure temperature using GPIB via an Agilent 82357A USB/GPIB converter
    this converter is hooked up to the raspberry pi and the LakeShore Model 
    218 temperature monitor.
'''

def T_measure():

    return temp

In [ ]:
''' 
    measure gallons per minute using an analog-digital converter
    ADC measures the voltage across a resistive load and converts
    it into a digital signal for the raspberry pi to read in mV's
    ADS1115 documentation at https://docs.circuitpython.org/projects/ads1x15/en/latest/ 
'''
def GPM_measure(resistor_load=1):
    # Create the I2C bus
    i2c = busio.I2C(board.SCL, board.SDA)

    # Create the ADC object using the I2C bus
    ads = ADS.ADS1115(i2c)

    # Create single-ended input on channel 0
    chan = AnalogIn(ads, ADS.P0)

    # Create differential input between channel 0 and 1
    #chan = AnalogIn(ads, ADS.P0, ADS.P1)

    print("{:>5}\t{:>5}".format('raw', 'v'))
    
    ''' perform measurement using ADC and gpib '''
    # use try-except to allow pi to continue working even if error is raised

    # take measurement and print
    print("{:>5}\t{:>5.3f}".format(chan.value, chan.voltage))

    ''' convert measurement '''
    # convert mA measurement to flowrate
    ### range of device is from 4mA to 20mA
    ### gpm is from ASP to AEP, by default 0.00 to 5.28 gpm

    
    #changed SSP and SEP to be function parameters
    #SSP = 4     # signal start point [mV]
    #SEP = 20    # signal end point [mV]
    ASP = 0.00   # analog start point [gpm]
    AEP = 5.28   # analog end point [gpm]

    # shift data down by 4mV (SSP), then 
    # divide by 16mV (SEP-SSP) to get percentage. 
    # then convert to gpm using the meter's analog range
    percentage = (data-SSP)/(SEP-SSP)
    gpm = percentage*(AEP-ASP)

    return gpm

In [ ]:
'''
    send text messages on reading failure to the 
    phone numbers listed below in the phonebook.
'''
phonebook = {
                "Jorge" : '2403054216@tmomail.net',
                "Manuel" : '7203520897@tmomail.net',
                "Pete" : '3034781436@vtext.com',
            }

# create yagmail object to send messages 
yag = yagmail.SMTP('scefreezepi', 'lbvgeofynaxplpne')  #username, gmail app password. actual password is "nistscegroup"


def Alert_protocol(alert, temp_reading, flow_reading, recipients=phonebook):
    '''
        INPUTS
            alert -  specific message to be sent regarding alert
            temp_reading -  float returned from T_measure()
            flow_reading -  float returned from GPM_measure()
            recipients -  
        OUTPUTS
            none
    '''
    
    time_report = datetime.datetime.now().strftime("%H:%M:%S --- %h %d %Y")
    subject = "Freeze's Alarm Triggered:"
    contents = "Alert: {} --- Triggered at {}  --- Temperature: {} K --- Flow rate: {} GPM".format(alert, time_report, temp_reading, flow_reading)
    
    yag.send(debug_jorge, subject, contents)
    return 

Alert_protocol("VAL SUCKS", -1, -1, recipients=phonebook["Jorge"])  # test
time.sleep(600)
Alert_protocol("test reading", -1, -1, recipients=phonebook["Jorge"])  # test

In [ ]:
''' USEFUL OLD CODE
# this will be appended to the text file

# now save data to opened file

'''


'''
loop methodology
1) check flow reading
2) check temperature reading
3) report bad readings via text
4) log to file and wait x minutes
'''
    
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop') 
script_path = os.path.join(desktop, "ProSense_Flowmeter_Script\\")
data_path = os.path.join(script_path, "Data\\")
cwd = os.getcwd()
# create directory if script folder is missing
        if not os.path.exists(script_path):
            os.makedirs(script_path)
            
        # create directory if data folder is missing   
        if not os.path.exists(data_path):
            os.makedirs(data_path)
                              
        # create text file with today's date as name
        today = datetime.date.today()
        filename = today.strftime("%m_%d_%y") + ".txt"   # e.g. 09_13_22.txt
        try:
            with open(os.path.join(data_path, filename), 'a') as f:  # use append mode in case file exists already
                date = str(datetime.datetime.now().strftime("%D %I:%M:%S %p"))  #Mon-Day-Yr  Hr:Min:Sec am/pm
                f.write('Initializing flowrate data collection: ' + date + '\n')
                
        except FileNotFoundError:
            print("except: FileNotFoundError")
    
    
while True:
    
    ''' step (1) '''
    try: 
        temp_reading = T_measure() 
    except:
        temp_reading = -1
        print("Temperature reading failed.")
    
    
    ''' step (2) '''
    try:
        flow_reading = GPM_measure()
    except:
        flow_reading = -1
        print("Flow reading failed.")
    
    
    ''' step (3) '''
    if flow_reading == -1 or temp_reading == -1:  
        print("Activating alert protocol.")
        Alert_protocol(alert="Bad reading", flow_reading, temp_reading, recipients=phonebook["Jorge"])  # only I shall know my mistakes
    
    
    ''' step (4) '''
    try:
        with open(os.path.join(os.getcwd(), filename), 'a') as file:  # use append mode to... append to today's file
            timestamp = str(datetime.datetime.now().strftime("%I:%M:%S %p")) # Hr:Min:Sec am/pm
            data_measurement_string = "[{0}] {1:1.2f} GPM @ {2:1.5f} mV  \n".format(timestamp, flow_rate, data)

            print(data_measurement_string)  # record in console
            file.write(data_measurement_string)  # record in text file

    except FileNotFoundError:
        print("except: FileNotFoundError")
    
    time.sleep(1800)